In [36]:
import numpy as np
from scipy import ndimage, misc
import copy

from deap import base
from deap import creator
from deap import tools
import game


In [2]:
n_weights = 9
population_size = 10
print_options = 1

In [6]:
''' DEAP SETUP '''
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("GameInstance", game.Game)
creator.create("Individual", list, 
                fitness=creator.FitnessMin, 
                gameInstance=creator.GameInstance)

toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, n_weights)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("evaluate", evalAlgorithm)

/opt/anaconda3/envs/ML2/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/opt/anaconda3/envs/ML2/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'GameInstance' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/opt/anaconda3/envs/ML2/lib/python3.5/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [37]:
# Initialise individual
ind = toolbox.individual()

# Evolve game
for i in range(10):
    ind.gameInstance.move(np.random.randint(0,4))

In [40]:
''' F1 '''
# f1 : Feature f1 is 1, if tile with maximum value is on one of the corners in a 
# given state. Otherwise, it is 0.
def f1(individual):
    flat_grid = individual.gameInstance.grid.flatten()

    maxvalue = max(flat_grid)
    for i,value in enumerate(flat_grid):
        if value == maxvalue and i in [0,3,11,15]:
            return 1
    return 0

''' F2 '''
# Feature f2 is 1, if second or third highest value tile is next to highest
# value tile. Otherwise, it is 0.
def f2(individual):
    maxval = max(individual.gameInstance.grid.flatten())
    if maxval==2: return 0

    if maxval==4:
        maxvalues = [4,2]
    else:
        maxvalues = [maxval, maxval//2, maxval//4]

    for row in range(4):
        for column in range(4):
            val = individual.gameInstance.grid[row,column]
            if val == maxval:
                # # Check above
                # if column-1 >= 0 and individual.gameInstance.grid[row,column-1] in maxvalues:
                #     return 1

                # Check right
                if row+1 <= 3 and individual.gameInstance.grid[row+1,column] in maxvalues:
                    return 1

                # # Check below
                # if column+1 <= 3 and individual.gameInstance.grid[row,column+1] in maxvalues:
                #     return 1

                # Check left
                if row-1 >= 0 and individual.gameInstance.grid[row-1,column] in maxvalues:
                    return 1
    return 0

''' F3 '''
# Sum of total changes 
def f3(individual, newstate):
    return 16-sum(individual.gameInstance.grid.flatten() == newstate.flatten())

''' F4 '''
# Number of non-empty cells in the given state of the board.
def f4(individual):
    return sum([i for i in individual.gameInstance.grid.flatten() if i != 0])

''' F5 '''
# How many out of selected cells in the given state have equal neighbors.
def f5(individual):
    counter = 0
    for row in range(4):
        for column in range(4):
            val = individual.gameInstance.grid[row,column]
            # Check right
            if row+1 <= 3 and individual.gameInstance.grid[row+1,column] == val:
                counter += 1
            # Check left
            if row-1 >= 0 and individual.gameInstance.grid[row-1,column] == val:
                counter += 1
    return counter

In [ ]:
# The evaluation function
def evalAlgorithm(individual):
    
    return

In [ ]:
def main():
    
    # Build the individual
    ind1 = toolbox.individual()
    
    # Create the population
    population = toolbox.population(population_size)
    
    # Keep track of the no. of generations
    generations = 0
    
    # Begin evolution
    while generations < 2000:
                
        # Selection
        selected = toolbox.select(population, len(population))
        offspring = [toolbox.clone(ind) for ind in selected]
        
        # Mutation
        mutant = toolbox.clone(offspring)
        ind2, = tools.mutGaussian(mutant, mu=0.0, sigma=0.2, indpb=0.2)
        del mutant.fitness.values  

        # Crossover
        child1, child2 = [toolbox.clone(ind) for ind in zip(offspring[::2], offspring[1::2])]
        tools.cxBlend(child1, child2, 0.5)
        del child1.fitness.values
        del child2.fitness.values

    
        # Evaluate fitness 
        ind1.fitness.values = evalAlgorithm(ind1)
            
        pop[:] = offspring
        

        
    
        best = pop[np.argmin([toolbox.evaluate(x) for x in pop])]
        
        # Increment the generation counter
        generations += 1    
    
    return best
    

In [ ]:
run_algorithm = main()

In [ ]:
# Start 2048 game
game_sample = game.Game(print_game=True)

In [ ]:
# Move down
game_sample.move(0)